<a href="https://colab.research.google.com/github/ja-pa/ea-openai-evals/blob/main/OAI_evals_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook pro testovani evalu


In [ ]:
#@title OpenAI API key


openai_key = '' #@param {type:"string"}

# Setup
Tato cast stahne evals repo a nainstaluje potrebne python balicky

In [ ]:
!git clone https://github.com/openai/evals.git
!cd evals && git lfs fetch --all && git lfs pull
#!cat evals/evals/registry/data/logic/samples.jsonl
!cd evals && pip install -e .

import pandas as pd
import json
from google.colab import data_table

import os
import glob

def display_table_from_file(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            # Parse each line as a JSON object
            json_line = json.loads(line)

            # Extract 'content' from 'input' and 'ideal' fields
            content = json_line['input'][0]['content']
            ideal = json_line['ideal']

            # Append to data list
            data.append({'Content': content, 'Ideal': ideal})

    # Convert the list to a DataFrame
    df = pd.DataFrame(data)

    # Return the DataFrame
    return df

def get_latest_file(path, pattern):
    list_of_files = glob.glob(os.path.join(path, pattern))
    if not list_of_files:
        return None
    latest_file = max(list_of_files, key=os.path.getctime)
    return latest_file


#log_output="/tmp/evallogs/230530205047OSMJXQ5A_gpt-3.5-turbo_logic-fact.jsonl"
def print_log(log_output):
  with open(log_output,"r") as fp:
    for line in fp.readlines():
      json_object = json.loads(line)
      json_formatted_str = json.dumps(json_object, indent=2)
      print(json_formatted_str)

# Vyhodnoceni evalu

Spusti vyhodnoceni evalu pro logic-fact (viz https://github.com/openai/evals/blob/main/evals/registry/evals/logic.yaml)

U logic-fact je eval_type **cot_classify**

viz https://github.com/openai/evals/blob/main/docs/eval-templates.md
>"cot_classify" ("chain-of-thought then classify", i.e., reason then answer) expects that the parsable portion of the response (i.e., the portion containing the choice) will be at the end of the completion. We recommend this as the default as it typically provides the most accurate model-graded evaluations.





In [ ]:
!cd evals && OPENAI_API_KEY=$openai_key oaieval gpt-3.5-turbo logic-fact

Zobrazi evaly pro logic ve forme tabulky

In [ ]:
# Call the function with your file path and print the DataFrame
df = display_table_from_file('evals/evals/registry/data/logic/samples.jsonl')
display(df)


Zobrazi posledni vygenerovany log z **oaieval** pro logic-fact

In [ ]:

log_output=get_latest_file("/tmp/evallogs", "*gpt-3.5-turbo_logic-fact.jsonl")
print("Log name:", log_output)
print_log(log_output)